<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/notebook/Assembly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [274]:
!pip install autorch > log.txt
!pip install optuna > log.txt

In [275]:
import joblib
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autorch
from autorch.function import sp2wt
import optuna

# LOAD DATA

In [276]:
icg_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl')
c620_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl')
c660_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl')
t651_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/t651_col_names.pkl')
c670_c = joblib.load('/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl')

In [277]:
icg_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/icg_train.csv',index_col=0).dropna(axis=0)
c620_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c620_train.csv',index_col=0).dropna(axis=0)
c660_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c660_train.csv',index_col=0).dropna(axis=0)
c670_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/c670_train.csv',index_col=0).dropna(axis=0)
t651_df = pd.read_csv('/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/t651_train.csv',index_col=0).dropna(axis=0)
allidx = list(set(icg_df.index)&
      set(c620_df.index)&
      set(c660_df.index)&
      set(c670_df.index)&
      set(t651_df.index))
len(allidx)

1295

# INPUT

In [278]:
# icg
icg_input = icg_df.loc[allidx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[allidx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[allidx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[allidx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[allidx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[allidx,t651_c['x41']]

# OUTPUT

In [279]:
c620_op = c620_df.loc[allidx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[allidx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[allidx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[allidx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[allidx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[allidx,c670_c['distillate_x']+c670_c['bottoms_x']]

# CONFIG

In [280]:
config = {
      # c620
      'c620_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_G.pkl',
      'c620_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c620_F.pkl',
      'c620_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_min.pkl',
      'c620_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_op_max.pkl',

      # c660
      'c660_G':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_G.pkl',
      'c660_F':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c660_F.pkl',
      'c660_op_min':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_min.pkl',
      'c660_op_max':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_op_max.pkl',

      # c670
      'c670_M':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/c670.pkl',
      
      # col_names
      'icg_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/icg_col_names.pkl',
      'c620_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c620_col_names.pkl',
      'c660_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c660_col_names.pkl',
      'c670_col_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/col_names/c670_col_names.pkl',
      
      # Special column (0.9999 & 0.0001)
      'index_9999_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_9999.pkl',
      'index_0001_path':'/content/drive/MyDrive/台塑輕油案子/data/c620/cleaned/index_0001.pkl',

      # sp
      'c620_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c620_wt_always_same_split_factor_dict.pkl',
      'c660_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c660_wt_always_same_split_factor_dict.pkl',
      'c670_wt_always_same_split_factor_dict':'/content/drive/MyDrive/台塑輕油案子/data/c620/map_dict/c670_wt_always_same_split_factor_dict.pkl',

      'op_fix_model':'/content/drive/MyDrive/台塑輕油案子/data/c620/model/fix_model.pkl',
      
          }

# 輸入與輸出的對應關係

In [281]:
# 對應關係1
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_df['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
070-026,70.0,69.999985
127-026,70.0,69.999985
119-017,80.0,80.000000
107-017,80.0,79.999992
134-002,90.0,90.000000
...,...,...
075-023,70.0,70.000008
077-017,80.0,79.999992
068-020,70.0,70.000008
046-002,90.0,90.000000


In [282]:
#對應關係2
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1)*10000
b.name = 'c660_wt_NA_in_Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt_NA_in_Benzene_ppmw
070-026,920.0,920.003332
127-026,920.0,920.024789
119-017,920.0,920.006428
107-017,920.0,919.993891
134-002,980.0,979.990893
...,...,...
075-023,950.0,949.993929
077-017,920.0,920.002079
068-020,980.0,980.003211
046-002,980.0,980.005703


In [283]:
#對應關係3
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_df['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
070-026,5.000046,5.000046
127-026,2.500000,2.500000
119-017,10.000017,10.000017
107-017,9.999962,9.999962
134-002,9.999999,9.999999
...,...,...
075-023,9.999991,9.999991
077-017,10.000003,10.000003
068-020,9.999212,9.999212
046-002,9.999997,9.999997


In [284]:
class AllSystem(object):
  def __init__(self,config):
    
    # C620 部份
    self.c620_G = joblib.load(config['c620_G'])
    self.c620_F = joblib.load(config['c620_F'])
    self.c620_op_min = joblib.load(config['c620_op_min'])
    self.c620_op_max = joblib.load(config['c620_op_max'])
    
    # C660 部份
    self.c660_G = joblib.load(config['c660_G'])
    self.c660_F = joblib.load(config['c660_F'])
    self.c660_op_min = joblib.load(config['c660_op_min'])
    self.c660_op_max = joblib.load(config['c660_op_max'])

    # C670 部份
    self.c670_M = joblib.load(config['c670_M'])
    
    # 修正模組修正模擬模跟現場數據間的差異
    self.op_fix_model = joblib.load(config['op_fix_model'])

    # 欄位名稱(全名)
    self.icg_col = joblib.load(config['icg_col_path'])
    self.c620_col = joblib.load(config['c620_col_path'])
    self.c660_col = joblib.load(config['c660_col_path'])
    self.c670_col = joblib.load(config['c670_col_path'])
    
    # 其他資訊
    self.c620_wt_always_same_split_factor_dict = joblib.load(config['c620_wt_always_same_split_factor_dict'])
    self.c660_wt_always_same_split_factor_dict = joblib.load(config['c660_wt_always_same_split_factor_dict'])
    self.c670_wt_always_same_split_factor_dict = joblib.load(config['c670_wt_always_same_split_factor_dict'])
    self.index_9999 = joblib.load(config['index_9999_path'])
    self.index_0001 = joblib.load(config['index_0001_path'])
    
    # 廠區提供密度
    self.V615_density = 0.8626
    self.C820_density = 0.8731
    self.T651_density = 0.8749

    # 廠區通常調整欄位
    self.c620_op_col_can_change = ['Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC','Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC']
    self.c660_op_col_can_change = ['Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC','Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC']
  
  def inference(self,icg_input,c620_feed,t651_feed,real_data_mode=False):
    idx = icg_input.index
    # 計算c620_case
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    # 預測操作條件和分離係數
    c620_op = self.c620_G.predict(c620_case.join(c620_feed))
    
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op))
    # 計算組成
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    #c660
    #計算c660_feed
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density
    C820_Dist_ton = C820_Dist_m3*self.C820_density
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True)
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True)
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1)
    c660_mf = t651_mf + c620_mf_side
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf
    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    # 計算c660_case
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case'])
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values
    # 預測操作條件和分離係數
    c660_op = self.c660_G.predict(c660_case.join(c660_feed))
    
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op))
    # 計算組成
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    #c670
    # 計算c670_feed
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True)
    c670_mf = c620_mf_bot + c660_mf_bot
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    # 計算c670_upper_bf
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf'])
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    # 直接預測分離係數和操作條件
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    # 計算輸出組成
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    # 是否修正操作條件 for 現場數據
    if real_data_mode == False:
      return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
    
    if real_data_mode == True:
      # 有些欄位現場數據沒有
      c620_op_col = c620_op.drop(['Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c660_op_col = c660_op.drop(['Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c670_op_col = c670_op.drop(['Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      
      # 經過修正模組修正op
      op_pred = self.op_fix_model(torch.cat((
        torch.FloatTensor(c620_op[c620_op_col].values),
        torch.FloatTensor(c660_op[c660_op_col].values),
        torch.FloatTensor(c670_op[c670_op_col].values)),dim=1))
      op_pred = pd.DataFrame(op_pred.detach().numpy(),index=idx)
      
      # 新的op
      new_c620_op = op_pred.iloc[:,:8]
      new_c660_op = op_pred.iloc[:,8:16]
      new_c670_op = op_pred.iloc[:,-5:]
      new_c620_op.columns = c620_op_col
      new_c660_op.columns = c660_op_col
      new_c670_op.columns = c670_op_col
      
      # 更新op
      c620_op.update(new_c620_op)
      c660_op.update(new_c660_op)
      c670_op.update(new_c670_op)
      
      return c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op
  
  def recommend(self,icg_input,c620_feed,t651_feed,search_iteration=300,real_data_mode=False,auto_set_x0=False):
    # 先試算一遍
    idx = icg_input.index
    c620_wt,c620_op,c660_wt,c660_op,c670_wt,c670_op = self.inference(icg_input,c620_feed,t651_feed)
    
    # 是否要自動設置 70,980,10 , 否則的話使用者可以自行設定 例如,80,990,15
    if auto_set_x0 == True:
      icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'] = 70.0
      icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = 980.0
      icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = 10.0
      
    # c620 部份
    c620_case = pd.DataFrame(index=idx,columns=self.c620_col['case'])
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 1 : Receiver Temp_oC'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 3 : Benzene in Sidedraw_wt%'] = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values
    c620_op_col = c620_op.columns.tolist()
    
    # c620 目標函數 
    def c620_objective(trial):
      # 首先操作是可以調整的
      c620_op_opt_dict = {}
      for name in c620_op_col:
        # 但是廠區通常只調整溫度欄位
        if name in self.c620_op_col_can_change:
          c620_op_opt_dict[name] = trial.suggest_uniform(name,self.c620_op_min[name],self.c620_op_max[name])
        else:
        # 所以其他欄位就依照試算模式給的設定
          c620_op_opt_dict[name] = c620_op[name].values[0] 
      
      c620_op_opt = pd.DataFrame(c620_op_opt_dict,index=idx)

      # 這裡取代ICG功能 歷史數據中Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr範圍落在 10.25~0 在這個範圍內搜索即可
      dist_rate = trial.suggest_float('Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr',0,10.25)
      c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = dist_rate
      # 有了c620_case,c620_feed,c620_op_opt就可以計算c620_wt
      c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
      s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
      # 計算Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt% 誤差
      輸入端bz = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
      輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
      
      # bz 希望輸入輸出兩端數值一致
      loss1 = abs(輸入端bz - 輸出端bz) / 輸入端bz      
      
      # 鼓勵dist_rate接近於1e-8 愈小愈好但是不能為0
      loss2 = abs(dist_rate - 1e-8) / dist_rate 
      
      return loss1 + loss2**2
    
    # cma-es給定優化初始值x0
    x0 = {}
    for name in c620_op_col:
      x0[name] = c620_op[name].values[0]
    # Distillate Rate_m3/hr 從使用者輸入的數值當作初始值開始調整 調整範圍在 0~10.25 多數情況下從 0 往上遞增
    x0['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'].values[0]
    
    # cma-es 開始優化
    sampler = optuna.samplers.CmaEsSampler(x0=x0)
    study = optuna.create_study(sampler=sampler)
    study.optimize(c620_objective, n_trials = search_iteration)
    
    # 優化完畢給出可能是最優的參數
    params_df = pd.DataFrame(study.best_params,index=idx,columns = c620_op_col+['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
    
    # 最優的操作 操作條件裡面只有屬於self.c620_op_col_can_change會被替換掉
    c620_op_opt = c620_op.drop(self.c620_op_col_can_change,axis=1).join(params_df[c620_op_col][self.c620_op_col_can_change])
    # 最優的dist rate
    c620_case['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'] = params_df['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr']

    # 有了c620_case,c620_feed,c620_op_opt計算c620_wt
    c620_sp = self.c620_F.predict(c620_case.join(c620_feed).join(c620_op_opt))
    s1,s2,s3,s4 = c620_sp.iloc[:,:41].values,c620_sp.iloc[:,41:41*2].values,c620_sp.iloc[:,41*2:41*3].values,c620_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c620_feed,s1),sp2wt(c620_feed,s2),sp2wt(c620_feed,s3),sp2wt(c620_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c620_wt = pd.DataFrame(wt,index=idx,columns=self.c620_col['vent_gas_x']+self.c620_col['distillate_x']+self.c620_col['sidedraw_x']+self.c620_col['bottoms_x'])
    
    # 計算Benzene in C620 Sidedraw_wt% 輸入端和輸出端誤差
    輸入端bz = icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'].values[0]
    輸出端bz = c620_wt['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%'].values[0]
    bz_error = abs(輸入端bz-輸出端bz)
    
    # c660 部份
    V615_Btm_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from V615 Btm_m3/hr'].values.reshape(-1,1)
    C820_Dist_m3 = icg_input['Simulation Case Conditions_Feed Rate_Feed from C820 Dist_m3/hr'].values.reshape(-1,1)
    V615_Btm_ton = V615_Btm_m3*self.V615_density # 體積流量轉重量流量
    C820_Dist_ton = C820_Dist_m3*self.C820_density # 體積流量轉重量流量
    c620_feed_rate_ton = V615_Btm_ton+C820_Dist_ton # 兩股相加得到c620總入料重量流量
    c620_mf_side = np.sum(c620_feed_rate_ton*c620_feed.values*s3*0.01,axis=1,keepdims=True) # c620_side 流量
    c620_mf_bot = np.sum(c620_feed_rate_ton*c620_feed.values*s4*0.01,axis=1,keepdims=True) # c620_bott 流量
    t651_mf = (icg_input['Simulation Case Conditions_Feed Rate_Feed from T651_m3/hr']*self.T651_density).values.reshape(-1,1) # t651重量流量
    c660_mf = t651_mf + c620_mf_side # 兩股相加得到c660入料重量流量
    t651_mf_p ,c620_mf_side_p = t651_mf/c660_mf ,c620_mf_side/c660_mf # 兩股油源的占比百分比

    c660_feed = c620_wt[self.c620_col['sidedraw_x']].values*c620_mf_side_p + t651_feed.values*t651_mf_p #計算c660入料組成
    c660_feed = pd.DataFrame(c660_feed,index=idx,columns=self.c660_col['x41'])
    c660_case = pd.DataFrame(index=idx,columns=self.c660_col['case']) # 計算c660_case
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']
    c660_case['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'] = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']
    c660_op_col = c660_op.columns.tolist()
    
    # c660 目標函數
    def c660_objective(trial):
      # 首先操作是可以調整的
      c660_op_opt_dict = {}
      for name in c660_op_col:
        # 但是廠區通常只調整溫度欄位
        if name in ['Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC','Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC']:
          c660_op_opt_dict[name] = trial.suggest_uniform(name,self.c660_op_min[name],self.c660_op_max[name])
        else:
        # 所以其他欄位就依照試算模式給的設定
          c660_op_opt_dict[name] = c660_op[name].values[0]
      
      # c660優化操作條件dataframe
      c660_op_opt = pd.DataFrame(c660_op_opt_dict,index=idx)
      
      # 取代ICG功能 歷史數據中NA in Benzene_ppmw範圍落在 [800~980] 在這個範圍內搜索即可
      輸入端nainbz =  trial.suggest_float('Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw',800,980)
      # 搜索到一個可能比較優的NA in Benzene_ppmw 代入c660_case
      c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = 輸入端nainbz
      # 這裡設定輸入端tol為一個數值,可以是使用者給定或是開啟auto_set_x0 = True 自動設置為10
      輸入端tol = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]

      # 有了c660_case,c660_feed,c660_op_opt計算c660_wt
      c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
      s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
      w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
      wt = np.hstack((w1,w2,w3,w4))
      c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
      
      # 計算誤差 NA in Benzene_ppmw誤差 和 Toluene in Benzene_ppmw 誤差
      na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
      輸出端nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000
      輸出端tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
      
      # 這一項是多個成份的加總 希望輸入輸出兩端數值一致
      loss1 = abs(輸入端nainbz - 輸出端nainbz) / 輸入端nainbz
      
      # 這一項是單個成份簡稱tol 希望輸入輸出兩端數值一致
      loss2 = abs(輸入端tol - 輸出端tol) / 輸入端tol
      
      return loss1**2 + loss2
    
    # cma-es 優化初始值 x0
    x0 = {}
    for name in c660_op_col:
       x0[name] = c660_op[name].values[0]
    
    # NA in Benzene_ppmw 從使用者輸入的數值當作初始值開始調整 調整範圍在 980~800 多數情況下從980遞減
    x0['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    
    # cma-es 優化
    sampler = optuna.samplers.CmaEsSampler(x0=x0)
    study = optuna.create_study(sampler=sampler)
    study.optimize(c660_objective, n_trials=search_iteration)
    
    # 優化過的操作條件
    params_df = pd.DataFrame(study.best_params,index = idx,columns = c660_op_col+['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'])
    # 最優的操作 操作條件裡面只有屬於self.c660_op_col_can_change會被替換掉
    c660_op_opt = c660_op.drop(self.c660_op_col_can_change,axis=1).join(params_df[c660_op_col][self.c660_op_col_can_change])
    # 最優的nainbz
    c660_case['Benzene Column C660 Operation_Specifications_Spec 2 : NA in Benzene_ppmw'] = params_df['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'] = params_df['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']
    
    # 計算c660_wt
    c660_sp = self.c660_F.predict(c660_case.join(c660_feed).join(c660_op_opt))
    s1,s2,s3,s4 = c660_sp.iloc[:,:41].values,c660_sp.iloc[:,41:41*2].values,c660_sp.iloc[:,41*2:41*3].values,c660_sp.iloc[:,41*3:41*4].values
    w1,w2,w3,w4 = sp2wt(c660_feed,s1),sp2wt(c660_feed,s2),sp2wt(c660_feed,s3),sp2wt(c660_feed,s4)
    wt = np.hstack((w1,w2,w3,w4))
    c660_wt = pd.DataFrame(wt,index=idx,columns=self.c660_col['vent_gas_x']+self.c660_col['distillate_x']+self.c660_col['sidedraw_x']+self.c660_col['bottoms_x'])
    
    # 計算誤差 NA in Benzene_ppmw誤差 和 Toluene in Benzene_ppmw 誤差
    輸入端nainbz = icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'].values[0]
    輸入端tol = icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'].values[0]
    na_idx = [1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] 
    輸出端nainbz = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,na_idx].sum(axis=1).values[0]*10000
    輸出端tol = c660_wt['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%'].values[0]*10000
    nainbz_error = abs(輸入端nainbz-輸出端nainbz)
    tol_error = abs(輸入端tol-輸出端tol)
    
    # 打印優化結果
    print('bz_error:',bz_error)
    print('nainbz_error:',nainbz_error)
    print('tol_error:',tol_error)

    # c670 部份
    c660_mf_bot = np.sum(c660_mf*c660_feed.values*s4*0.01,axis=1,keepdims=True) # c660_bot 重量流量
    c670_mf = c620_mf_bot + c660_mf_bot # c620_bot 重量流量 加上 c660_bot 重量流量 得到c670總入料重量流量
    c620_mf_bot_p,c660_mf_bot_p = c620_mf_bot/c670_mf , c660_mf_bot/c670_mf # 兩股油源的百分比占比
    c670_feed = c620_wt[self.c620_col['bottoms_x']].values*c620_mf_bot_p + c660_wt[self.c660_col['bottoms_x']].values*c660_mf_bot_p # c670入料組成
    c670_feed = pd.DataFrame(c670_feed,index=idx,columns=self.c670_col['combined'])
    
    # c670 upper_bf 計算
    c670_bf = pd.DataFrame(index=idx,columns=self.c670_col['upper_bf']) 
    c620_bot_x = c620_wt[self.c620_col['bottoms_x']].values
    c660_bot_x = c660_wt[self.c660_col['bottoms_x']].values
    upper_bf = (c660_bot_x*c660_mf_bot)/(c620_bot_x*c620_mf_bot+c660_bot_x*c660_mf_bot)
    upper_bf = pd.DataFrame(upper_bf,index=idx,columns=self.c670_col['upper_bf'])
    upper_bf[list(set(self.index_9999)&set(upper_bf.columns))] = 0.9999
    upper_bf[list(set(self.index_0001)&set(upper_bf.columns))] = 0.0001
    
    # c670因為沒有指定wt數值要多少所以直接預測操作條件和分離係數即可
    c670_output = self.c670_M.predict(c670_feed.join(upper_bf))
    c670_sp,c670_op_opt = c670_output.iloc[:,:41*2],c670_output.iloc[:,41*2:]    
    
    # 計算c670_wt
    s1 = c670_sp[self.c670_col['distillate_sf']].values
    s2 = c670_sp[self.c670_col['bottoms_sf']].values
    w1 = sp2wt(c670_feed,s1)
    w2 = sp2wt(c670_feed,s2)
    c670_wt = np.hstack((w1,w2))
    c670_wt = pd.DataFrame(c670_wt,index = idx,columns=self.c670_col['distillate_x']+self.c670_col['bottoms_x'])
    
    # 是否修正操作條件 for 現場數據
    if real_data_mode == False:
      return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error
    
    if real_data_mode == True:
      # 有些欄位現場數據沒有 因此drop掉
      c620_op_col = c620_op.drop(['Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c660_op_col = c660_op.drop(['Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      c670_op_col = c670_op.drop(['Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr',
                                 'Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr'],
                                 axis=1).columns.tolist()
      
      # 經過修正模組修正op
      op_pred = self.op_fix_model(torch.cat((
        torch.FloatTensor(c620_op_opt[c620_op_col].values),
        torch.FloatTensor(c660_op_opt[c660_op_col].values),
        torch.FloatTensor(c670_op_opt[c670_op_col].values)),dim=1))  
      op_pred = pd.DataFrame(op_pred.detach().numpy(),index=idx)
      
      # 新的op
      new_c620_op = op_pred.iloc[:,:8]
      new_c660_op = op_pred.iloc[:,8:16]
      new_c670_op = op_pred.iloc[:,-5:]
      new_c620_op.columns = c620_op_col
      new_c660_op.columns = c660_op_col
      new_c670_op.columns = c670_op_col
      
      # 更新op
      c620_op_opt.update(new_c620_op)
      c660_op_opt.update(new_c660_op)
      c670_op_opt.update(new_c670_op)
      
      return c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error

# 試算模式測試 觀察重點 準

In [285]:
f = AllSystem(config)
idx = np.random.choice(allidx,size=100,replace=False,p=None)
# minibatch input 
icg_input = icg_input.loc[idx]
c620_feed = c620_feed.loc[idx]
t651_feed = t651_feed.loc[idx]
# minibatch output 
c620_op = c620_op.loc[idx]
c620_wt = c620_wt.loc[idx]
c660_op = c660_op.loc[idx]
c660_wt = c660_wt.loc[idx]
c670_op = c670_op.loc[idx]
c670_wt = c670_wt.loc[idx]
c620_wt_,c620_op_,c660_wt_,c660_op_,c670_wt_,c670_op_ = f.inference(icg_input,c620_feed,t651_feed)
f.c670_M.show_metrics(c620_wt,c620_wt_,e=2e-2)

,R2,MSE,MAPE
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.843264,0.00247925,2.1292
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.867666,0.529922,1.50287
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.0689506,0.120898,0.509056
Tatoray Stripper C620 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.857096,0.215898,4.38311
...,...,...,...
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999193,2.21992e-06,0.327379
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999178,4.09272e-07,0.327377
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Tatoray Stripper C620 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [286]:
f.c670_M.show_metrics(c620_op,c620_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.999839,2.35876e-10,0.00137851
Density_Vent Gas Production Rate and Composition,0.945345,1.08277e-06,0.18677
Density_Distillate Production Rate and Composition,0.852768,1.57072e-05,0.436053
Density_Sidedraw Production Rate and Composition,0.994825,5.75391e-09,0.00625448
Density_Bottoms Production Rate and Composition,0.99668,5.42208e-11,0.000701748
Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,0.976266,3.36139,0.875984
Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.984346,0.0200823,0.756741
Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.980184,0.0342546,0.860228
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,0.991288,0.0108609,0.0450049
Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC,0.987834,0.00930732,0.0401297


In [287]:
f.c670_M.show_metrics(c660_wt,c660_wt_,e=2e-2)

,R2,MSE,MAPE
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Methane_wt%,0.879294,0.0267076,11.432
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Ethane_wt%,0.911542,3.86183,6.91044
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_Propane_wt%,0.640429,2.46226,5.54269
Benzene Column C660 Operation_Vent Gas Production Rate and Composition_n-Butane_wt%,0.906764,1.22272,13.595
...,...,...,...
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.980712,2.61196e-25,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.981765,3.00644e-31,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Benzene Column C660 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [288]:
f.c670_M.show_metrics(c660_op,c660_op_,e=2e-2)

,R2,MSE,MAPE
Density_Feed Properties,0.985168,1.14764e-08,0.00865234
Density_Vent Gas Production Rate and Composition,0.94782,1.8847e-05,0.667986
Density_Distillate (Benzene Drag) Production Rate and Composition,0.94698,4.45183e-06,0.202724
Density_Sidedraw (Benzene )Production Rate and Composition,0.96712,3.16598e-12,0.000166391
Density_Bottoms Production Rate and Composition,0.993854,5.0432e-10,0.0020242
Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,0.975128,25.972,3.08527
Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.973854,0.212326,2.82523
Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.975657,0.20897,2.94359
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,0.974082,0.00143657,0.0347084
Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC,0.968044,0.0312298,0.142217


In [289]:
f.c670_M.show_metrics(c670_wt,c670_wt_,e=2e-2)

,R2,MSE,MAPE
Toluene Column C670 Operation_Distillate Production Rate and Composition_Hydrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Methane_wt%,0,2.56003e-10,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Ethane_wt%,0,1.49475e-07,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_Propane_wt%,0,1.3029e-07,NaN
Toluene Column C670 Operation_Distillate Production Rate and Composition_n-Butane_wt%,0,6.10643e-47,NaN
...,...,...,...
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Pentylbenzene_wt%,0.999999,2.09015e-08,0.0174976
Toluene Column C670 Operation_Bottoms Production Rate and Composition_n-Hexylbenzene_wt%,0.999999,3.89923e-09,0.0176286
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Nitrogen_wt%,1,0,NaN
Toluene Column C670 Operation_Bottoms Production Rate and Composition_Oxygen_wt%,1,0,NaN


In [290]:
f.c670_M.show_metrics(c670_op,c670_op_,e=2e-2)

,R2,MSE,MAPE
Density_Distillate Production Rate and Composition,0.995191,4.20103e-12,0.000182653
Density_Bottoms Production Rate and Composition,0.995451,8.62313e-10,0.00251364
Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,0.979443,13.5263,1.15654
Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,0.981372,0.174813,1.09539
Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,0.980452,0.175631,1.09139
Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,0.952573,0.0026211,0.0234251
Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC,0.988225,0.0323212,0.0622379
AVG,0.981815,1.98739,0.490239


# 確認三項條件有滿足

In [291]:
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
110-026,70.0,70.000008
060-020,70.0,70.000008
041-011,80.0,80.000015
062-017,80.0,79.999992
096-023,70.0,70.000015
...,...,...
102-008,90.0,89.999992
017-008,90.0,90.000015
067-008,90.0,89.999992
010-020,70.0,70.000008


In [292]:
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt NA in NA in Benzene_ppmw
110-026,920.0,920.034444
060-020,980.0,979.989987
041-011,980.0,979.995972
062-017,920.0,920.000239
096-023,830.0,829.997735
...,...,...
102-008,860.0,860.003811
017-008,890.0,890.196732
067-008,920.0,920.025513
010-020,980.0,980.003770


In [293]:
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt[['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
110-026,5.000000,5.000000
060-020,9.999996,9.999996
041-011,10.000005,10.000005
062-017,5.000000,5.000000
096-023,10.000007,10.000007
...,...,...
102-008,5.000000,5.000000
017-008,9.999785,9.999785
067-008,2.500000,2.500000
010-020,9.999997,9.999997


# 推薦模式測試觀察重點調幅(op_opt - op)以及nainbz是否有提高distrate是否有降低

In [294]:
f = AllSystem(config)
idx = 5
print(idx)
icg_input = icg_input.iloc[[idx]]
c620_feed = c620_feed.iloc[[idx]]
t651_feed = t651_feed.iloc[[idx]]
# minibatch output 
c620_op = c620_op.iloc[[idx]]
c620_wt = c620_wt.iloc[[idx]]
c660_op = c660_op.iloc[[idx]]
c660_wt = c660_wt.iloc[[idx]]
c670_op = c670_op.iloc[[idx]]
c670_wt = c670_wt.iloc[[idx]]

print(icg_input['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%'])
print(icg_input['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw'])
print(icg_input['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
print(icg_input['Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])

5
137-026    70.0
Name: Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%, dtype: float64
137-026    920.0
Name: Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw, dtype: float64
137-026    10.000004
Name: Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw, dtype: float64
137-026    0.01
Name: Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr, dtype: float64


In [295]:
c620_wt,c620_op_opt,c660_wt,c660_op_opt,c670_wt,c670_op_opt,bz_error,nainbz_error,tol_error = f.recommend(icg_input,c620_feed,t651_feed,auto_set_x0=True)

[I 2021-04-27 06:06:39,794] A new study created in memory with name: no-name-6f1a367d-0dd0-4b20-852e-d0b3adb61553
[I 2021-04-27 06:06:39,886] Trial 0 finished with value: 1.0309216878984173 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 177.57709807500933, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 192.84669114718156, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 6.843733152811992}. Best is trial 0 with value: 1.0309216878984173.
[I 2021-04-27 06:06:39,966] Trial 1 finished with value: 1.039407292662057 and parameters: {'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC': 176.63588526175147, 'Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC': 193.2391442027155, 'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr': 0.060448244364232365}. Best is trial 0 with value: 1.030921687898417

bz_error: 0.00011389303367081993
nainbz_error: 25.614805648515244
tol_error: 0.00019890276186451672


# 確認三項條件有滿足

In [296]:
bz_error,nainbz_error,tol_error

(0.00011389303367081993, 25.614805648515244, 0.00019890276186451672)

In [297]:
a = icg_input[['Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%']]
b = c620_wt[['Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%']]
a.join(b)

,Simulation Case Conditions_Spec 1 : Benzene in C620 Sidedraw_wt%,Tatoray Stripper C620 Operation_Sidedraw Production Rate and Composition_Benzene_wt%
137-026,70.0,69.999886


In [298]:
a = icg_input[['Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw']]
b = c660_wt.filter(regex='Side').filter(regex='wt%').iloc[:,[1,2,3,4,5,6,8,9,11,13,14,15,20,22,29] ].sum(axis=1)*10000
b.name = 'c660_wt NA in NA in Benzene_ppmw'
a.join(b)

,Simulation Case Conditions_Spec 2 : NA in Benzene_ppmw,c660_wt NA in NA in Benzene_ppmw
137-026,980.0,954.385194


In [299]:
a = icg_input[['Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw']]
b = c660_wt[['Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%']]*10000
a.join(b)

,Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw,Benzene Column C660 Operation_Sidedraw (Benzene )Production Rate and Composition_Toluene_wt%
137-026,10.0,9.999801


# 確認操作條件有足夠大的調幅

In [300]:
c620_op_opt - c620_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate Production Rate and Composition,Density_Sidedraw Production Rate and Composition,Density_Bottoms Production Rate and Composition,Tatoray Stripper C620 Operation_Yield Summary_Reflux Rate_m3/hr,Tatoray Stripper C620 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 14 (Control)_oC,Tatoray Stripper C620 Operation_Column Temp Profile_C620 Tray 34 (Control)_oC
137-026,0.00001,-0.001078,0.0039,-0.000024,-0.000014,0.367538,0.016959,0.036184,-1.035162,0.072604


In [301]:
c660_op_opt - c660_op

,Density_Feed Properties,Density_Vent Gas Production Rate and Composition,Density_Distillate (Benzene Drag) Production Rate and Composition,Density_Sidedraw (Benzene )Production Rate and Composition,Density_Bottoms Production Rate and Composition,Benzene Column C660 Operation_Yield Summary_Reflux Rate_m3/hr,Benzene Column C660 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 6 (SD & Control)_oC,Benzene Column C660 Operation_Column Temp Profile_C660 Tray 23 (Control)_oC
137-026,0.000075,0.004751,-0.001505,-4.881698e-07,-7.799027e-07,3.733704,0.238716,0.258264,0.053441,0.612161


In [302]:
c670_op_opt - c670_op

,Density_Distillate Production Rate and Composition,Density_Bottoms Production Rate and Composition,Toluene Column C670 Operation_Yield \nSummary_Reflux Rate_m3/hr,Toluene Column C670 Operation_Heat Duty_Condenser Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Heat Duty_Reboiler Heat Duty_Mkcal/hr,Toluene Column C670 Operation_Column Temp Profile_C670 Tray 24 (Control)_oC,Toluene Column C670 Operation_Column Temp Profile_C670 Btm Temp (Control)_oC
137-026,-0.000002,-0.000013,1.730408,0.171803,0.145745,-0.056869,0.121613


# 確定以下三點誤差足夠小

In [303]:
bz_error

0.00011389303367081993

In [304]:
nainbz_error

25.614805648515244

In [305]:
tol_error

0.00019890276186451672

# 保存樣本

In [306]:
idx = icg_input.index
print(idx)

Index(['137-026'], dtype='object')


In [307]:
# icg
icg_input = icg_df.loc[allidx,icg_c['x']]
icg_input = icg_input.join(c620_df.loc[allidx,'Tatoray Stripper C620 Operation_Specifications_Spec 2 : Distillate Rate_m3/hr'])
icg_input = icg_input.join(c660_df.loc[allidx,'Benzene Column C660 Operation_Specifications_Spec 3 : Toluene in Benzene_ppmw'])
icg_input = icg_input.join(c620_df.loc[allidx].filter(regex='Receiver Temp'))

# c620
c620_feed = c620_df.loc[allidx,c620_c['x41']]

# t651
t651_feed = t651_df.loc[allidx,t651_c['x41']]

In [308]:
c620_op = c620_df.loc[allidx,c620_c['density']+c620_c['yRefluxRate']+c620_c['yHeatDuty']+c620_c['yControl']]
c620_wt = c620_df.loc[allidx,c620_c['vent_gas_x']+c620_c['distillate_x']+c620_c['sidedraw_x']+c620_c['bottoms_x']]
c660_op = c660_df.loc[allidx,c660_c['density']+c660_c['yRefluxRate']+c660_c['yHeatDuty']+c660_c['yControl']]
c660_wt = c660_df.loc[allidx,c660_c['vent_gas_x']+c660_c['distillate_x']+c660_c['sidedraw_x']+c660_c['bottoms_x']]
c670_op = c670_df.loc[allidx,c670_c['density']+c670_c['yRefluxRate']+c670_c['yHeatDuty']+c670_c['yControl']]
c670_wt = c670_df.loc[allidx,c670_c['distillate_x']+c670_c['bottoms_x']]

In [309]:
demo = {
      # input
      'icg_input':icg_input.loc[idx],
      'c620_feed':c620_feed.loc[idx],
      't651_feed':t651_feed.loc[idx],
      # output
      'c620_op':c620_op.loc[idx],
      'c620_wt':c620_wt.loc[idx],
      'c660_op':c660_op.loc[idx],
      'c660_wt':c660_wt.loc[idx],
      'c670_op':c670_op.loc[idx],
      'c670_wt':c670_wt.loc[idx],
      }

In [310]:
import joblib
joblib.dump(demo,"/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/demo/demo.pkl']

# 保存模組

In [311]:
joblib.dump(f,"/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl")

['/content/drive/MyDrive/台塑輕油案子/data/c620/model/allsystem.pkl']